<a href="https://colab.research.google.com/github/VladimirB23/Project_2/blob/main/prolongation_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# импортируем библиотеки
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# создадим функцию для загузки данных через по ID
def load_from_gdrive(file_id: str) -> pd.DataFrame:
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url)

# и сразу создадим переменные с дф
prolongations = load_from_gdrive("1n2CIsPZYOIqMfV7b3aUtp1efc9U96xVt")
financial_data = load_from_gdrive("1r3grFMz_k-XuWhRNLOAZCoLDdVYg2C7X")

In [ ]:
# сделаем предварительный анализ (EDA)
# для начала посмотрим на первые строки каждого дф
prolongations.head()

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [ ]:
financial_data.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [ ]:
# Копия (чтобы ничего не сломать)
df = financial_data.copy()

# 1) Покажем текущие имена колонок — убедимся, как точно называется "Причина дубля"
print("Колонки:", df.columns.tolist())

# 2) Определим месячные колонки по наличию года в названии (2022/2023/2024)
month_cols = [c for c in df.columns if re.search(r'20\d{2}', str(c))]
print("Найдено month_cols (порядок важен):", month_cols)

# 3) Функция очистки одной серии (месяц)
def clean_money_series(s):
    s = s.astype(str).str.strip()
    s = s.str.replace('в ноль', '0', case=False, regex=True)   # 'в ноль' -> 0
    s = s.str.replace('\u00A0', '', regex=False)              # NBSP
    s = s.str.replace(' ', '', regex=False)                   # обычные пробелы
    s = s.str.replace(',', '.', regex=False)                  # десятичный разделитель
    # удалить любые символы кроме цифр, точки и минуса
    s = s.str.replace(r'[^0-9\.\-]', '', regex=True)
    s = s.replace('', np.nan)                                 # пустые → NaN
    return pd.to_numeric(s, errors='coerce')

# 4) Применяем только к month_cols
for c in month_cols:
    df[c] = clean_money_series(df[c]).fillna(0.0)

# 5) Диагностика: покажем 1) сколько строк с ненулевыми отгрузками; 2) первые 8 примеров
mask_nonzero = (df[month_cols].abs().sum(axis=1) > 0)
print("Всего строк:", len(df), "| строк с ненулевыми отгрузками:", int(mask_nonzero.sum()))

cols_show = ['id','Причина дубля'] if 'Причина дубля' in df.columns else ['id']
cols_show += month_cols[:6]  # для компактного просмотра - первые 6 месяцев
display(df.loc[mask_nonzero, cols_show].head(8))

# 6) Сохраняем в financial_clean (только если всё ок)
financial_clean = df.copy()

Колонки: ['id', 'Причина дубля', 'Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024', 'Account']
Найдено month_cols (порядок важен): ['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024']
Всего строк: 451 | строк с ненулевыми отгрузками: 419


,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023
0,42,NaN,36220.0,0.0,0.0,0.0,0.0,0.0
4,665,NaN,10000.0,0.0,0.0,0.0,0.0,0.0
5,637,NaN,38045.0,0.0,0.0,0.0,0.0,0.0
7,578,NaN,82800.0,0.0,0.0,0.0,0.0,0.0
9,592,первая часть оплаты,92302.0,0.0,0.0,0.0,0.0,0.0
10,592,вторая часть оплаты,12340.0,0.0,0.0,0.0,0.0,0.0
11,666,NaN,110000.0,0.0,0.0,0.0,0.0,0.0
13,704,первая часть оплаты,35365.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# определим размер таблиц
print("Размер prolongations:", prolongations.shape)
print("Размер financial_data:", financial_data.shape)

# проверим специальные значения в financial_data
special_values = ['стоп','end','в ноль','stop','END','STOP']
# пройдемся по каждому и посчитаем кол-во
for val in special_values:
    mask = financial_data.apply(lambda x: x.astype(str).str.lower().str.contains(val)).any(axis=1)
    count = mask.sum()
    print(f"Количество строк с '{val}': {count}")

# проверим пересечения по id в двух дф
ids_prol = set(prolongations['id'])
ids_fin  = set(financial_data['id'])
print("Уникальных проектов только в prolongations:", len(ids_prol - ids_fin))
print("Уникальных проектов только в financial_data:", len(ids_fin - ids_prol))
print("Совпадающих проектов:", len(ids_prol & ids_fin))


Размер prolongations: (477, 3)
Размер financial_data: (451, 19)
Количество строк с 'стоп': 71
Количество строк с 'end': 1
Количество строк с 'в ноль': 54
Количество строк с 'stop': 0
Количество строк с 'END': 0
Количество строк с 'STOP': 0
Уникальных проектов только в prolongations: 0
Уникальных проектов только в financial_data: 1
Совпадающих проектов: 313


In [ ]:
# подготовим данные так, чтобы легко посчитать "последний месяц проект"
# объединим оба дф по id и добавим к каждому проекту информацию о его окончании
df_merged = financial_clean.merge(
    prolongations,
    on = 'id',
    how = 'inner' # оставляем только проекты, которые есть в обоех таблицах
)
print("Размер объединенного датафрейма", df_merged.shape)
display(df_merged.head())

Размер объединенного датафрейма (725, 21)


,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,...,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account,month,AM
0,42,NaN,36220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
1,657,первая часть оплаты,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
2,657,вторая часть оплаты,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
3,594,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович
4,665,NaN,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Васильев Артем Александрович,ноябрь 2022,Васильев Артем Александрович


In [ ]:
# проверим сколько уникальных id без дублей
print("Уникальных id в объединенном дф:", df_merged['id'].nunique())

Уникальных id в объединенном дф: 313


In [ ]:
# определяем колонки-месяцы (они содержат суммы отгрузок)
month_cols = [col for col in df_merged.columns if '20' in col]
# и уберем всё таки дубликаты
df_unique = (
    df_merged
    .groupby(['id', 'AM', 'month'], as_index=False)[month_cols] # группируем по id проекта, аккаунт-менеджеру и последнему месяцу реализации
    .sum() # суммируем значения отгрузки по месяцам (если были дубли – они аккуратно сложатся)
)

# ещё раз проверим размерность и уникальность
print("Размер после очистки от дублей:", df_unique.shape)
print("Уникальных id:", df_unique['id'].nunique())
print("Количество менеджеров:", df_unique['AM'].nunique())

Размер после очистки от дублей: (474, 19)
Уникальных id: 313
Количество менеджеров: 10


In [ ]:
# Подготовка по проектам: ship_last, ship_m1, ship_m2 и пробная сводка

# Переименуем колонки месяцев в нижний регистр, чтобы совпадали со значениями в month
df_unique = df_unique.rename(columns={c: c.lower() for c in month_cols})
# 1) установим порядок в month_cols и нормализуем month в df_unique (уберем пробелы)
month_cols = [c for c in df_unique.columns if ('2022' in c) or ('2023' in c) or ('2024' in c)]
month_cols = [c.strip() for c in month_cols]
df_unique['month'] = df_unique['month'].astype(str).str.strip()

# 2) карта "следующий" и "через 2" по индексу (без работы с датами)
next1 = {m: (month_cols[i+1] if i+1 < len(month_cols) else None) for i, m in enumerate(month_cols)}
next2 = {m: (month_cols[i+2] if i+2 < len(month_cols) else None) for i, m in enumerate(month_cols)}

# 3) добавим колонки
df = df_unique.copy()
df['ship_last'] = 0.0
df['ship_m1']   = 0.0
df['ship_m2']   = 0.0

# 4) векторное заполнение: для каждой возможной "month" берем значения из соответствующих колонок
for m in month_cols:
    mask = df['month'] == m
    if not mask.any():
        continue
    # ship_last — сумма в колонке с именем last_month
    df.loc[mask, 'ship_last'] = df.loc[mask, m].astype(float).fillna(0.0)
    # ship_m1 — следующая колонка по порядку (если есть)
    c1 = next1.get(m)
    if c1:
        df.loc[mask, 'ship_m1'] = df.loc[mask, c1].astype(float).fillna(0.0)
    # ship_m2 — колонка через 2 шага (если есть)
    c2 = next2.get(m)
    if c2:
        df.loc[mask, 'ship_m2'] = df.loc[mask, c2].astype(float).fillna(0.0)

# 5) правило "в ноль" для К1: если в M+1 = 0, то для числителя используем ship_last (только если ship_last>0)
# (иначе — 0)
df['prol1_amt'] = np.where(df['ship_m1'] != 0, df['ship_m1'],
                           np.where(df['ship_last'] != 0, df['ship_last'], 0.0))

# 6) для К2: числитель — ship_m2 у тех проектов, у которых в M+1 нет отгрузок (ship_m1 == 0)
# здесь просто сохраняем ship_m2; фильтрация по ship_m1 будет при агрегации
df['prol2_amt'] = df['ship_m2'].astype(float).fillna(0.0)

# 7) диагностика — короткая сводка и первые 8 строк таблицы с ключевыми колонками
total = len(df)
have_last = (df['ship_last'] != 0).sum()
have_m1   = (df['ship_m1'] != 0).sum()
have_m2   = (df['ship_m2'] != 0).sum()

print(f"Всего проектов в df: {total}")
print(f"Проектов с ненулевым ship_last: {have_last}; с ненулевым ship_m1: {have_m1}; с ненулевым ship_m2: {have_m2}")

cols_show = ['id', 'AM', 'month', 'ship_last', 'ship_m1', 'ship_m2', 'prol1_amt', 'prol2_amt']
display(df[cols_show].head(8))

# 8) сохраняем результат в переменную для следующих шагов
df_prepared = df  # далее будем использовать df_prepared

Всего проектов в df: 474
Проектов с ненулевым ship_last: 394; с ненулевым ship_m1: 201; с ненулевым ship_m2: 202


,id,AM,month,ship_last,ship_m1,ship_m2,prol1_amt,prol2_amt
0,15,Иванова Мария Сергеевна,апрель 2023,138158.00,138158.00,102433.75,138158.00,102433.75
1,15,Иванова Мария Сергеевна,декабрь 2022,439280.00,102433.75,102433.75,102433.75,102433.75
2,15,Иванова Мария Сергеевна,июнь 2023,102433.75,0.00,0.00,102433.75,0.00
3,15,Иванова Мария Сергеевна,март 2023,102433.75,138158.00,138158.00,138158.00,138158.00
4,15,Иванова Мария Сергеевна,февраль 2023,102433.75,102433.75,138158.00,102433.75,138158.00
5,16,Иванова Мария Сергеевна,декабрь 2022,0.00,0.00,0.00,0.00,0.00
6,31,Васильев Артем Александрович,декабрь 2022,55100.00,0.00,44775.00,55100.00,44775.00
7,39,Иванова Мария Сергеевна,декабрь 2022,137700.00,149206.50,149206.50,149206.50,149206.50


In [ ]:
# расчёт CR1/CR2 (помесячно и год)

# карта месяцев (приводим к lower)
RU_MONTHS = {
    "январь": 1, "февраль": 2, "март": 3, "апрель": 4,
    "май": 5, "июнь": 6, "июль": 7, "август": 8,
    "сентябрь": 9, "октябрь": 10, "ноябрь": 11, "декабрь": 12
}

def normalize_month_label(s):
    """Принимает строку вроде 'апрель 2023' или 'Апрель 2023' или 'Ноябрь 2022'
       Возвращает 'MM.YYYY' (например '04.2023') или None если не распознал."""
    if pd.isna(s):
        return None
    s = str(s).strip()
    # Иногда есть лишние пробелы, т.е. берем первые и последние токены
    parts = s.split()
    if len(parts) < 2:
        return None
    month_name = parts[0].lower()
    year = parts[-1]
    # безопасно: если год содержит не-цифры — отказ
    if not year.isdigit():
        return None
    month_num = RU_MONTHS.get(month_name)
    if not month_num:
        return None
    return f"{month_num:02d}.{year}"

# 0) Нормализуем исходный столбец month в df_prepared (создадим вспомогательное поле month_norm)
df = df_prepared.copy()  # работаем на копии, чтобы не портить исходные данные
df['month_norm'] = df['month'].apply(normalize_month_label)

# 1) Собираем порядок месячных колонок (из заголовков колонок с '20**') и нормализуем порядок
month_cols = [c for c in df.columns if re.search(r'20\d{2}', str(c))]
# Преобразуем month_cols (названия столбцов) в нормализованные метки и отсортируем хронологически
month_col_pairs = []
for col in month_cols:
    norm = normalize_month_label(col)
    if norm is not None:
        # norm -> 'MM.YYYY', конвертим в sortable ключ YYYYMM
        y, m = norm.split(".")[1], norm.split(".")[0]
        key = int(y) * 100 + int(m)
        month_col_pairs.append((col, norm, key))

# сортировка по ключу (год*100 + месяц)
month_col_pairs = sorted(month_col_pairs, key=lambda x: x[2])
sorted_months_norm = [t[1] for t in month_col_pairs]   # список норм. месяцев в хронолог. порядке

# оставим месяцы 2023
months_2023 = [m for m in sorted_months_norm if m.endswith('.2023')]

# контейнеры результатов
rows_cr1_total = []
rows_cr1_am = []
rows_cr2_total = []
rows_cr2_am = []

# 2) Проходим по каждому месяцу 2023 и считаем K1 и K2 (используем норм. метки)
for m_norm in months_2023:
    i = sorted_months_norm.index(m_norm)
    # K1: проекты, завершившиеся в предыдущем месяце (prev_m_norm)
    if i == 0:
        continue
    prev_m_norm = sorted_months_norm[i - 1]

    # выбираем проекты, у которых поле month_norm == prev_m_norm
    base_k1 = df[df['month_norm'] == prev_m_norm]

    den1 = base_k1['ship_last'].sum()
    num1 = base_k1['prol1_amt'].sum()
    k1 = (num1 / den1) if den1 else np.nan

    rows_cr1_total.append({
        'month': m_norm,
        'den1': den1,
        'num1': num1,
        'CR1': k1,
        'projects': len(base_k1)
    })

    # по менеджерам
    if not base_k1.empty:
        grp1 = base_k1.groupby('AM', dropna=False).agg(
            den1=('ship_last', 'sum'),
            num1=('prol1_amt', 'sum'),
            projects=('id', 'count')
        ).reset_index()
        grp1['CR1'] = grp1.apply(lambda r: (r['num1'] / r['den1']) if r['den1'] else np.nan, axis=1)
        grp1['month'] = m_norm
        rows_cr1_am.append(grp1)
    else:
        # если база пуста — добавим пустую запись для консистентности (необязательно)
        pass

    # K2: для месяца m — база = проекты, завершившиеся в m-2 и у которых ship_m1 == 0
    if i < 2:
        continue
    prev2_norm = sorted_months_norm[i - 2]
    base_k2 = df[(df['month_norm'] == prev2_norm) & (df['ship_m1'] == 0)]

    den2 = base_k2['ship_last'].sum()
    num2 = base_k2['prol2_amt'].sum()
    k2 = (num2 / den2) if den2 else np.nan

    rows_cr2_total.append({
        'month': m_norm,
        'den2': den2,
        'num2': num2,
        'CR2': k2,
        'projects': len(base_k2)
    })

    if not base_k2.empty:
        grp2 = base_k2.groupby('AM', dropna=False).agg(
            den2=('ship_last', 'sum'),
            num2=('prol2_amt', 'sum'),
            projects=('id', 'count')
        ).reset_index()
        grp2['CR2'] = grp2.apply(lambda r: (r['num2'] / r['den2']) if r['den2'] else np.nan, axis=1)
        grp2['month'] = m_norm
        rows_cr2_am.append(grp2)

# 3) Собираем итоговые датафреймы
df_cr1_total = pd.DataFrame(rows_cr1_total)
df_cr1_am = pd.concat(rows_cr1_am, ignore_index=True) if rows_cr1_am else pd.DataFrame()
df_cr2_total = pd.DataFrame(rows_cr2_total)
df_cr2_am = pd.concat(rows_cr2_am, ignore_index=True) if rows_cr2_am else pd.DataFrame()

# 4) Убираем "без а/м" везде сразу (если встречается буквальная строка)
if not df_cr1_am.empty:
    mask = ~df_cr1_am['AM'].astype(str).str.strip().str.lower().isin(['', 'nan', 'без а/м'])
    df_cr1_am = df_cr1_am[mask].copy()
if not df_cr2_am.empty:
    mask = ~df_cr2_am['AM'].astype(str).str.strip().str.lower().isin(['', 'nan', 'без а/м'])
    df_cr2_am = df_cr2_am[mask].copy()

# 5) Годовые агрегаты (Total)
k1_den_year = df_cr1_total['den1'].sum() if not df_cr1_total.empty else 0
k1_num_year = df_cr1_total['num1'].sum() if not df_cr1_total.empty else 0
CR1_year_total = (k1_num_year / k1_den_year) if k1_den_year else np.nan

k2_den_year = df_cr2_total['den2'].sum() if not df_cr2_total.empty else 0
k2_num_year = df_cr2_total['num2'].sum() if not df_cr2_total.empty else 0
CR2_year_total = (k2_num_year / k2_den_year) if k2_den_year else np.nan

df_year_total = pd.DataFrame([{
    'CR1_den': k1_den_year, 'CR1_num': k1_num_year, 'CR1': CR1_year_total,
    'CR2_den': k2_den_year, 'CR2_num': k2_num_year, 'CR2': CR2_year_total
}])

# 6) Годовые по менеджерам
df_cr1_am_year = df_cr1_am.groupby('AM', dropna=False).agg(
    den1=('den1', 'sum'), num1=('num1', 'sum')
).reset_index()
df_cr1_am_year['CR1'] = df_cr1_am_year.apply(lambda r: (r['num1'] / r['den1']) if r['den1'] else np.nan, axis=1)

df_cr2_am_year = df_cr2_am.groupby('AM', dropna=False).agg(
    den2=('den2', 'sum'), num2=('num2', 'sum')
).reset_index()
df_cr2_am_year['CR2'] = df_cr2_am_year.apply(lambda r: (r['num2'] / r['den2']) if r['den2'] else np.nan, axis=1)

# --- Быстрая проверка результатов (печатаем сводку) ---
print("=== Проверка ===")
print("Порядок месяцев (нормализованный):", sorted_months_norm)
print("Месяцы 2023 (используемые):", months_2023)
print("df_cr1_total размер:", df_cr1_total.shape)
print("df_cr2_total размер:", df_cr2_total.shape)
print("df_cr1_am размер:", df_cr1_am.shape)
print("df_cr2_am размер:", df_cr2_am.shape)
print("df_year_total:")
display(df_year_total.head(3))

# Теперь можно безопасно экспортировать df_cr1_total, df_cr2_total, df_cr1_am, df_cr2_am, df_year_total, df_cr1_am_year, df_cr2_am_year
# В них поле 'month' уже в формате MM.YYYY (например '01.2023').

=== Проверка ===
Порядок месяцев (нормализованный): ['11.2022', '12.2022', '01.2023', '02.2023', '03.2023', '04.2023', '05.2023', '06.2023', '07.2023', '08.2023', '09.2023', '10.2023', '11.2023', '12.2023', '01.2024', '02.2024']
Месяцы 2023 (используемые): ['01.2023', '02.2023', '03.2023', '04.2023', '05.2023', '06.2023', '07.2023', '08.2023', '09.2023', '10.2023', '11.2023', '12.2023']
df_cr1_total размер: (12, 5)
df_cr2_total размер: (12, 5)
df_cr1_am размер: (72, 6)
df_cr2_am размер: (62, 6)
df_year_total:


,CR1_den,CR1_num,CR1,CR2_den,CR2_num,CR2
0,34362627.23,33810417.97,0.98393,14913472.87,1097390.58,0.073584


In [ ]:
# экспортируем результаты в Excel

# создадим Excel с несколькими вкладками
with pd.ExcelWriter("report_CR.xlsx", engine="openpyxl") as writer:
    # помесячные CR1 и CR2 (Total)
    df_cr1_total.to_excel(writer, sheet_name="CR1_monthly_total", index=False)
    df_cr2_total.to_excel(writer, sheet_name="CR2_monthly_total", index=False)

    # помесячные по менеджерам
    df_cr1_am.to_excel(writer, sheet_name="CR1_monthly_AM", index=False)
    df_cr2_am.to_excel(writer, sheet_name="CR2_monthly_AM", index=False)

    # годовые итоги
    df_year_total.to_excel(writer, sheet_name="Year_total", index=False)
    df_cr1_am_year.to_excel(writer, sheet_name="CR1_year_AM", index=False)
    df_cr2_am_year.to_excel(writer, sheet_name="CR2_year_AM", index=False)

print("✅ Файл report_CR.xlsx успешно создан. Можно загрузить его или Excel/Google Sheets и сделать визуализации вручную")

✅ Файл report_CR.xlsx успешно создан. Можно загрузить его или Excel/Google Sheets и сделать визуализации вручную


In [ ]:
# можно скачать сырой отчет к себе на локальный диск
from google.colab import files
files.download("report_CR.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# либо можем загрузить файл автоматически в Google Sheets с переименованными полями для рук-ля
# установка и авторизация (нужно предоставить разрешение на авторизацию)
!pip install --quiet gspread gspread_dataframe

from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# создаём или открываем таблицу
SHEET_NAME = "Отчёт по пролонгации"
try:
    sh = gc.open(SHEET_NAME)
    print(f"Открыта существующая таблица: {SHEET_NAME}")
except gspread.SpreadsheetNotFound:
    sh = gc.create(SHEET_NAME)
    print(f"Создана новая таблица: {SHEET_NAME}")

# читаемые названия вкладок
rename_tabs = {
    "CR1_monthly_total": "CR1 по месяцам",
    "CR2_monthly_total": "CR2 по месяцам",
    "CR1_monthly_AM": "CR1 по месяцам (менеджеры)",
    "CR2_monthly_AM": "CR2 по месяцам (менеджеры)",
    "Year_total": "Итоги за год (отдел)",
    "CR1_year_AM": "CR1 за год (менеджеры)",
    "CR2_year_AM": "CR2 за год (менеджеры)"
}

# словарь для читаемых названий колоно
rename_columns = {
    "CR1_monthly_total": {
        "month": "Месяц",
        "den1": "Сумма базы CR1",
        "num1": "Сумма пролонгаций (м+1)",
        "CR1": "Коэф. пролонгации 1 мес.",
        "projects": "Кол-во проектов"
    },
    "CR2_monthly_total": {
        "month": "Месяц",
        "den2": "Сумма базы CR2",
        "num2": "Сумма пролонгаций (м+2)",
        "CR2": "Коэф. пролонгации 2 мес.",
        "projects": "Кол-во проектов"
    },
    "CR1_monthly_AM": {
        "AM": "Менеджер",
        "month": "Месяц",
        "den1": "Сумма базы CR1",
        "num1": "Сумма пролонгаций (м+1)",
        "CR1": "Коэф. пролонгации 1 мес.",
        "projects": "Кол-во проектов"
    },
    "CR2_monthly_AM": {
        "AM": "Менеджер",
        "month": "Месяц",
        "den2": "Сумма базы CR2",
        "num2": "Сумма пролонгаций (м+2)",
        "CR2": "Коэф. пролонгации 2 мес.",
        "projects": "Кол-во проектов"
    },
    "Year_total": {
        "CR1_den": "Сумма базы CR1 (год)",
        "CR1_num": "Сумма пролонгаций (год, м+1)",
        "CR1": "Годовой CR1",
        "CR2_den": "Сумма базы CR2 (год)",
        "CR2_num": "Сумма пролонгаций (год, м+2)",
        "CR2": "Годовой CR2"
    },
    "CR1_year_AM": {
        "AM": "Менеджер",
        "den1": "Сумма базы CR1 (год)",
        "num1": "Сумма пролонгаций (год, м+1)",
        "CR1": "Годовой CR1"
    },
    "CR2_year_AM": {
        "AM": "Менеджер",
        "den2": "Сумма базы CR2 (год)",
        "num2": "Сумма пролонгаций (год, м+2)",
        "CR2": "Годовой CR2"
    }
}

# список датафреймов для экспорта
dfs = {
    "CR1_monthly_total": df_cr1_total,
    "CR2_monthly_total": df_cr2_total,
    "CR1_monthly_AM": df_cr1_am,
    "CR2_monthly_AM": df_cr2_am,
    "Year_total": df_year_total,
    "CR1_year_AM": df_cr1_am_year,
    "CR2_year_AM": df_cr2_am_year
}

# экспортируем
for key, df in dfs.items():
    df_export = df.copy()
    # переименуем колонки
    df_export = df_export.rename(columns=rename_columns.get(key, {}))
    # читаемое название вкладки
    sheet_title = rename_tabs.get(key, key)

    # удаляем старый лист, если есть
    try:
        worksheet = sh.worksheet(sheet_title)
        sh.del_worksheet(worksheet)
    except gspread.WorksheetNotFound:
        pass

    # создаём новый лист и заливаем данные
    worksheet = sh.add_worksheet(title=sheet_title, rows=str(len(df_export)+1), cols=str(len(df_export.columns)))
    set_with_dataframe(worksheet, df_export)
    print(f"✅ Загружен лист: {sheet_title}")

print(f"📊 Готово! Проверь Google Sheets: {SHEET_NAME}")

Открыта существующая таблица: Отчёт по пролонгации
✅ Загружен лист: CR1 по месяцам
✅ Загружен лист: CR2 по месяцам
✅ Загружен лист: CR1 по месяцам (менеджеры)
✅ Загружен лист: CR2 по месяцам (менеджеры)
✅ Загружен лист: Итоги за год (отдел)
✅ Загружен лист: CR1 за год (менеджеры)
✅ Загружен лист: CR2 за год (менеджеры)
📊 Готово! Проверь Google Sheets: Отчёт по пролонгации


In [ ]:
from datetime import datetime

# Добавим мета-информацию
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
meta_df = pd.DataFrame({
    "Поле": ["Источник данных", "Дата обновления"],
    "Значение": ["financial_data + prolongations", now]
})

# Если лист meta уже есть → удаляем
try:
    worksheet = sh.worksheet("meta")
    sh.del_worksheet(worksheet)
except:
    pass

# Создаём заново
worksheet = sh.add_worksheet(title="meta", rows="100", cols="20")
set_with_dataframe(worksheet, meta_df)

📑 Документация к отчёту по пролонгации договоров

1. Входные данные

В качестве исходных данных использовались проекты, завершённые в течение 2023 года.\
Для каждого проекта были доступны следующие поля:\
 • id — уникальный идентификатор проекта,\
 • AM — аккаунт-менеджер, закреплённый за проектом,\
 • month — месяц завершения проекта,\
 • ship_last — сумма отгрузки в последний месяц реализации проекта,\
 • ship_m1 — сумма отгрузки в первый месяц после завершения проекта,\
 • prol1_amt — сумма пролонгации в первый месяц после завершения проекта,\
 • prol2_amt — сумма пролонгации во второй месяц после завершения проекта.

2. Основные коэффициенты

В компании используются два коэффициента пролонгации:\
 • CR1 (коэффициент пролонгации 1 мес.)\
CR1 = Сумма пролонгированных проектов в (m+1) / Сумма отгрузки завершившихся проектов в (m)\
 • CR2 (коэффициент пролонгации 2 мес.)
CR2 = Сумма пролонгированных проектов в (m+2) \ Сумма отгрузки завершившихся в (m) без пролонгации в (m+1)\
Где:\
 • den1 — база для CR1 (сумма отгрузки завершившихся проектов в месяце m),\
 • num1 — числитель для CR1 (сумма пролонгаций этих проектов в месяце m+1),\
 • den2 — база для CR2 (сумма отгрузки проектов, завершившихся в m-1 и не пролонгированных в m),\
 • num2 — числитель для CR2 (сумма пролонгаций этих проектов в месяце m+1).

3. Расчёты

3.1. Ежемесячные показатели по отделу

Для каждого месяца были рассчитаны:\
 • den1, num1, CR1, количество проектов — для первого месяца пролонгации;\
 • den2, num2, CR2, количество проектов — для второго месяца пролонгации.\
\
Результаты собраны в таблицах:\
 • CR1 по месяцам\
 • CR2 по месяцам

3.2. Ежемесячные показатели по менеджерам

Для каждого аккаунт-менеджера и месяца рассчитаны:\
 • Сумма базы (den1, den2),\
 • Сумма пролонгаций (num1, num2),\
 • Количество проектов,\
 • Коэффициенты CR1 и CR2.

Результаты собраны в таблицах:\
 • CR1 по месяцам (менеджеры)\
 • CR2 по месяцам (менеджеры)

  3.3. Итоги за год

  По отделу
\
Агрегация по всем месяцам:\
 • Сумма базы CR1 и CR2,\
 • Сумма пролонгаций CR1 и CR2,\
 • Годовые коэффициенты CR1 и CR2.

Собрано в таблице Итоги за год (отдел).

По менеджерам

Агрегация по каждому аккаунт-менеджеру за весь год:\
 • Сумма базы CR1 и CR2,\
 • Сумма пролонгаций CR1 и CR2,\
 • Годовые коэффициенты CR1 и CR2.

Собрано в таблицах:\
 • CR1 за год (менеджеры)\
 • CR2 за год (менеджеры)

4. Визуализации в DataLens\
Ссылка на Дашборд: https://datalens.yandex/33aukc9gsjrso\
  На основе расчётов были подготовлены:\
 • Линейные графики динамики CR1 и CR2 по месяцам,\
 • Комбинированные графики CR1 и CR2 по менеджерам (бар + линия отдела),\
 • Сводные таблицы по менеджерам с ключевыми метриками,\
 • Итоговая таблица по отделу,\
 • Пай-чарты с долями менеджеров в пролонгациях (CR1 и CR2).